In [6]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.distributions.normal import Normal
from torch.autograd import Variable
    
def softplus(rho):
    # convert from rho -> sigma
    return torch.log(1 + torch.exp(rho))     

class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_weights = 0
        
        self.weight = None # Parameter(torch.FloatTensor(in_features, out_features))
        self.num_weights += in_features * out_features
        
        if bias:
            self.bias = None # Parameter(torch.FloatTensor(out_features))
            self.num_weights += out_features
        else:
            self.register_parameter('bias', None)
                    
    def init_weights(self, weights):
        self.weight = weights[:self.in_features * self.out_features].reshape(self.in_features, self.out_features)
        if self.bias:
            self.bias = weights[self.in_features * self.out_features:]
        
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.layers = [self.gc1, self.gc2]
        self.num_weights = sum(layer.num_weights for layer in self.layers)
        self.dropout = dropout
        
    def kernel(self, x):
        ## Standard RBF kernel: x -> exp(-x*x)
        return torch.exp(-x*x)
    
    def init_params(self):
        return torch.rand(self.num_weights * 2, requires_grad=True) # stored as [mu1, mu2, rho1, rho2]
    
    def init_weights(self, var_params):
        ## Returns log_prob of weights, prior_prob of weights
        w_dist = Normal(var_params[:self.num_weights], softplus(var_params[self.num_weights:]))
        p_dist = Normal(torch.zeros(self.num_weights), torch.ones(self.num_weights) * 5)
        weights = w_dist.rsample()
        
        counter = 0
        for layer in self.layers:
            layer.init_weights(weights[counter:counter+layer.num_weights])
            counter += layer.num_weights
            
        return w_dist.log_prob(weights), p_dist.log_prob(weights)
        
    def forward(self, x, adj):
        x = self.kernel(self.gc1(x, adj))
        # x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)


In [2]:
from utils2 import load_data, load_adv_data

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [11]:
import time

def bbb(adj, features, labels, optimizer, idx_train, idx_val, model, params, epochs=140, samples=10, seed=42):
    torch.manual_seed(seed)
    
    for i in range(epochs):
        model.train()
        ## Clear gradients
        optimizer.zero_grad()
        # print(params.grad)
        
        total_ll = 0.0
        total_wl = 0.0
        total_wp = 0.0
        
        for j in range(samples):
                        
            ## Init weights
            weight_ll, weight_prior = model.init_weights(params)
            total_wl += weight_ll.sum() / model.num_weights
            total_wp += weight_prior.sum() / model.num_weights

            ## Forward pass, compute log_likelihood
            pred = model(features, adj)
            model_ll = - F.nll_loss(pred[idx_train], labels[idx_train])
            total_ll += model_ll

            ## Compute loss
            ELBO = - model_ll - (weight_prior.sum() - weight_ll.sum()) / model.num_weights
            # Why divide by weights twice? 

            ## Compute gradients
            ELBO.backward()
        
        if i % 50 == 0:
            total_ELBO = total_ll + total_wp - total_wl
            print("Train :: Epoch {0:d} | ELBO: {1:.3f} | LL: {2:.3f} ".format(i + 1, total_ELBO / samples, total_ll / samples))
        
        ## Update parameters
        # print("OLD", params[:5])
        optimizer.step()    
        # print("NEW", params[:5])
        
def val(adj, features, labels, idx_train, idx_val, model, params, samples=300, seed=42):
    torch.manual_seed(seed)
    model.eval() # This could be a problem
    
    train_ll = 0.0
    val_ll = 0.0
    train_acc = 0.0
    val_acc = 0.0

    for j in range(samples):
        ## Forward pass
        _ = model.init_weights(params)
        pred = model(features, adj)
        y_hat = torch.argmax(pred, dim=1)
        
        ## collect log_likelihood statistics
        train_ll += - F.nll_loss(pred[idx_train], labels[idx_train])
        val_ll += - F.nll_loss(pred[idx_val], labels[idx_val])
        
        ## collect accuracy statistics
        train_acc += accuracy(y_hat.numpy(), labels.numpy(), idx_train)
        val_acc += accuracy(y_hat.detach().numpy(), labels.numpy(), idx_val)
            
    print("Val   :: Train LL: {0:.3f} | Val LL: {1:.3f} | Train Accuracy: {2:.3f} | Val Accuracy: {3:.3f} ".format(
        train_ll / samples, val_ll / samples, train_acc / samples, val_acc / samples))

def accuracy(pred, true, idx):
    ## Outputs accuracy on given index
    assert(len(pred) == len(true))
    return np.sum((pred == true)[idx]) / len(true[idx])

class Counter:
    def __init__(self, types):
        self.values = {}
        for t in types:
            self.values[t] = []
            
    def avg(self, t):
        return np.mean(self.values[t])
    
    def update(self, xs, types):
        for i, x in enumerate(xs):
            self.values[types[i]].append(x)
            
    def print_all(self):
        for t in self.values.keys():
            print(t, self.avg(t))

nodes = idx_test[:50]
start = time.time()
counts = ['train_acc', 'val_acc', 'test_acc', 'adv_acc']
results = Counter(counts)

for node in nodes:
    model = GCN(nfeat=1433,
                nhid=16,
                nclass=7,
                dropout=0.5)
    params = model.init_params()
    optimizer = optimizer = optim.Adam([params], lr=2e-2, weight_decay=5e-4)

    adj, features, labels, idx_train, idx_val, idx_test = load_adv_data(node)

    bbb(adj, features, labels, optimizer, idx_train, idx_val, model, params, epochs=250, samples=10, seed=42)
    model.eval()
    _ = model.init_weights(params)
    pred = model(features, adj)
    pred = torch.argmax(pred, dim=1).numpy()
    print(pred)

    train_acc = accuracy(pred, labels.numpy(), idx_train)
    val_acc = accuracy(pred, labels.numpy(), idx_val)
    test_acc = accuracy(pred, labels.numpy(), idx_test)
    adv_acc = accuracy(pred, labels.numpy(), [node])
    results.update([train_acc, val_acc, test_acc, adv_acc], counts)

end = time.time()  
print("Time Taken", end-start)
results.print_all()


Loading cora dataset...
Added:  [2158 1136 1870] Removed:  [2444 1767  563]
Train :: Epoch 1 | ELBO: -6.135 | LL: -4.968 
Train :: Epoch 51 | ELBO: -3.253 | LL: -2.093 
Train :: Epoch 101 | ELBO: -2.512 | LL: -1.346 
Train :: Epoch 151 | ELBO: -2.064 | LL: -0.898 
Train :: Epoch 201 | ELBO: -1.697 | LL: -0.525 
[3 5 6 ... 4 1 3]
Loading cora dataset...
Added:  [ 344  183 2643] Removed:  [503 496 174]
Train :: Epoch 1 | ELBO: -6.968 | LL: -5.800 
Train :: Epoch 51 | ELBO: -3.281 | LL: -2.120 
Train :: Epoch 101 | ELBO: -2.608 | LL: -1.441 
Train :: Epoch 151 | ELBO: -2.019 | LL: -0.851 
Train :: Epoch 201 | ELBO: -1.666 | LL: -0.493 
[3 5 6 ... 4 1 3]
Loading cora dataset...
Added:  [ 784 1179 2315] Removed:  [2536  565  294]
Train :: Epoch 1 | ELBO: -6.968 | LL: -5.800 
Train :: Epoch 51 | ELBO: -3.281 | LL: -2.120 
Train :: Epoch 101 | ELBO: -2.608 | LL: -1.441 
Train :: Epoch 151 | ELBO: -2.018 | LL: -0.851 
Train :: Epoch 201 | ELBO: -1.666 | LL: -0.493 
[3 5 6 ... 4 1 3]
Loading co

Train :: Epoch 201 | ELBO: -1.667 | LL: -0.493 
[3 5 6 ... 4 1 3]
Loading cora dataset...
Added:  [ 201 1764 1689] Removed:  [1419 1425  522]
Train :: Epoch 1 | ELBO: -6.968 | LL: -5.800 
Train :: Epoch 51 | ELBO: -3.281 | LL: -2.121 
Train :: Epoch 101 | ELBO: -2.609 | LL: -1.441 
Train :: Epoch 151 | ELBO: -2.019 | LL: -0.851 
Train :: Epoch 201 | ELBO: -1.666 | LL: -0.493 
[3 5 6 ... 4 1 3]
Loading cora dataset...
Added:  [2398 2408] Removed:  [427 747]
Train :: Epoch 1 | ELBO: -6.967 | LL: -5.800 
Train :: Epoch 51 | ELBO: -3.281 | LL: -2.120 
Train :: Epoch 101 | ELBO: -2.608 | LL: -1.440 
Train :: Epoch 151 | ELBO: -2.018 | LL: -0.851 
Train :: Epoch 201 | ELBO: -1.666 | LL: -0.493 
[3 5 6 ... 4 1 3]
Loading cora dataset...
Added:  [2359 1210 1152] Removed:  [1968  747 2606]
Train :: Epoch 1 | ELBO: -6.967 | LL: -5.800 
Train :: Epoch 51 | ELBO: -3.281 | LL: -2.120 
Train :: Epoch 101 | ELBO: -2.608 | LL: -1.441 
Train :: Epoch 151 | ELBO: -2.019 | LL: -0.851 
Train :: Epoch 201 